# 4. Exploratory Data Analysis

**탐색적 데이터 분석을 통해 데이터를 통달해봅시다.** with [Titanic Data](https://www.kaggle.com/c/titanic)

0. 라이브러리 준비
1. 분석의 목적과 변수 확인
2. 데이터 전체적으로 살펴보기
3. 데이터의 개별 속성 파악하기

## 0. 라이브러리 준비

In [ ]:
## 라이브러리 불러오기

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

%matplotlib inline

kaggle competitions download -c titanic

unzip titanic.zip -d titanic

In [ ]:
## 타이타닉 데이터 가져오기 

titanic_df = pd.read_csv("./titanic/train.csv")


## 1. 분석의 목적과 변수 확인

In [ ]:
titanic_df.head(3)

In [ ]:
titanic_df.dtypes

## 2. 데이터 전체적으로 살펴보기

In [ ]:
## 데이터 전체 정보를 얻는 함수 : .describe()

titanic_df.describe()

In [ ]:
## 상관계수 확인!

titanic_df.corr()

#상관성 : Aup, Bup
#인과성 : A -> B

In [ ]:
### 결측치 확인 안에 null 있는 갯수

titanic_df.isnull().sum()

## 3. 데이터의 개별 속성 파악하기

### 1. Survied Column

In [ ]:
## 생존자, 사망자 명수 

print("sum : ",titanic_df['Survived'].sum())
print("count : ",titanic_df['Survived'].value_counts())

In [ ]:
## 생존자수와 사망자수를 Barplot으로 그려보기 sns.countplot()

sns.countplot(x='Survived', data=titanic_df)
plt.show()

### 2.Pclass

In [ ]:
# Pclass에 따른 인원 파악   
titanic_df[['Pclass','Survived']].groupby(['Pclass']).count()

In [ ]:
# 생존자 인원

titanic_df[['Pclass','Survived']].groupby(['Pclass']).sum()


In [ ]:
# 생존자 비율

titanic_df[['Pclass','Survived']].groupby(['Pclass']).mean()


In [ ]:
# 히트맵 활용
sns.heatmap(titanic_df[['Pclass','Survived']].groupby(['Pclass']).mean())
plt.show()

### 3.Sex

In [ ]:
titanic_df.groupby(['Sex','Survived'])['Survived'].count()

In [ ]:
# sns.catplot

sns.catplot(x='Sex', col='Survived', kind='count', data=titanic_df)
plt.show()

### 4. Age

In [ ]:
## 결측치 존재 조심

### 데이터의 그룹했을때 결과들을 볼수있다
titanic_df.describe()['Age']

In [ ]:
## Survived 1, 0 과 Age의 경향성
# figure 도면 => axit x축 y축 틀 => plot 값
# kdeplot 밀도함수

### plt.subplot (가로 몇개, 세로몇개# Survived 1, 0 과 Age의 경향성
# # figure 도면 => axit x축 y축 틀 => plot 값
# kdeplot 커널 밀도함수

### plt.subplot (가로 몇개, 세로몇개, 도면크기(가로, 세로) )
# 여기 fig 는 안씀 사이즈 알려주는거 같은데 
fig, ax =plt.subplots(1,1, figsize=(10, 5))

sns.kdeplot(x = titanic_df[titanic_df.Survived == 1]['Age'], ax=ax)
sns.kdeplot(x = titanic_df[titanic_df.Survived == 0]['Age'], ax=ax)

plt.legend(['Survive', 'Dead'])

plt.show()

### Appendix 1. Sex + Pclass vs Survived

In [ ]:
## 티켓의 등급에 따른 생존률을 보여준다
## 저기 짝대기는 다른 속성에따른 오차범위? 같은거 같다
sns.catplot(x='Pclass', y='Survived',  kind='point', data=titanic_df)

In [ ]:
## hue 속성을 하나더 추가해서 상관관계보는것

sns.catplot(x='Pclass', y='Survived', hue='Sex',  kind='point', data=titanic_df)

### Appendix 2.Age + Pclass

In [ ]:
## Age graph with Pclass
titanic_df['Age'][titanic_df.Pclass == 1].plot(kind='kde')
titanic_df['Age'][titanic_df.Pclass == 2].plot(kind='kde')
titanic_df['Age'][titanic_df.Pclass == 3].plot(kind='kde')

plt.legend(['1st class','2nd class','3rd class'])
plt.show()

## Mission : It's Your Turn!

### 1. 본문에서 언급된 Feature를 제외하고 유의미한 Feature를 1개 이상 찾아봅시다.

- Hint : Fare? Sibsp? Parch?

### 2. [Kaggle](https://www.kaggle.com/datasets)에서 Dataset을 찾고, 이 Dataset에서 유의미한 Feature를 3개 이상 찾고 이를 시각화해봅시다.

함께 보면 좋은 라이브러리 document
- [numpy]()
- [pandas]()
- [seaborn]()
- [matplotlib]()